### **환경 설정**

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:15:15tcmalloc: large alloc 1147494400 bytes == 0x39ef8000 @  0x7f29f3a54615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.2 MB/s eta 0:13:40tcmalloc: large alloc 1434370048 bytes == 0x7e54e000 @  0x7f29f3a54615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████           | 1336.2 MB 1.4 MB/s eta 0:08:23tcmalloc: large alloc 1792966656 bytes =

In [3]:
from mmcv import collect_env
collect_env()

{'CUDA available': True,
 'CUDA_HOME': '/usr/local/cuda',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'GPU 0': 'Tesla P100-PCIE-16GB',
 'MMCV': '1.5.0',
 'MMCV CUDA Compiler': '11.1',
 'MMCV Compiler': 'GCC 7.3',
 'NVCC': 'Cuda compilation tools, release 11.1, V11.1.105',
 'OpenCV': '4.1.2',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-genco

In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.24.1
11.1
GCC 7.3


In [17]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


### **데이터셋 다운로드**

In [5]:
# Download dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="MY_API_KEY")
project = rf.workspace("sketch").project("sketch-detection")
dataset = project.version(2).download("coco")

     |████████████████████████████████| 145 kB 7.0 MB/s 
     |████████████████████████████████| 178 kB 59.3 MB/s 
     |████████████████████████████████| 1.1 MB 67.4 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 138 kB 73.8 MB/s 
     |████████████████████████████████| 596 kB 62.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=5ceac066185a4e1911e22988f7ec1f45451e318e502a0a0437ced5c06e46739e
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=2f2cfde2d11c38f37e2b5504cf4d556ff925eb2d3641adb69dc7a96bcd9d9d50
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wget

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to sketch-detection-2 in coco:: 100%|██████████| 3309/3309 [00:02<00:00, 1280.94it/s]


### **TOOD**

<img src="https://user-images.githubusercontent.com/12907710/145400075-e08191f5-8afa-4335-9b3b-27926fc9a26e.png" width=500/>

In [26]:
from mmcv import Config
from mmdet.apis import set_random_seed

cfg = Config.fromfile('./configs/tood/tood_r101_fpn_dconv_c3-c5_mstrain_2x_coco.py')

print(cfg.pretty_text)

dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 480), (1333, 800)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
     

In [30]:
# config 설정

data_root = '/content/mmdetection/sketch-detection-2/'
ann_file = '_annotations.coco.json'


cfg.data_root = data_root

cfg.data.train.ann_file = data_root + 'train/' + ann_file
cfg.data.train.img_prefix = data_root + 'train/'
cfg.data.train.classes = ('button', 'checkbox_checked', 'checkbox_unchecked',
                            'chip', 'data_table', 'dropdown_menu',
                            'floating_action_button', 'image', 'radiobutton_checked',
                            'radiobutton_unchecked', 'slider', 'switch_disabled',
                            'switch_enabled', 'text_field', 'text_area')

cfg.data.val.ann_file = data_root + 'valid/' + ann_file
cfg.data.val.img_prefix = data_root + 'valid/'
cfg.data.val.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.data.test.ann_file = data_root + 'test/' + ann_file
cfg.data.test.img_prefix = data_root + 'test/'
cfg.data.test.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.optimizer.lr = 0.0025

cfg.model.bbox_head.num_classes = 15

cfg.runner.max_epochs = 30
cfg.max_epochs = 30

cfg.work_dir = 'working/tood'

cfg.seed = 0

cfg.gpu_ids = [0]
cfg.device = 'cuda'

cfg.log_config.interval = 100
cfg.lr_config.warmup = None
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [31]:
print(cfg.pretty_text)

# config 파일 저장
with open("tood.py","w") as f:
  f.write(cfg.pretty_text)

!cp tood.py /content/drive/MyDrive/model/tood

dataset_type = 'CocoDataset'
data_root = '/content/mmdetection/sketch-detection-2/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 480), (1333, 800)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomF

In [ ]:
# 학습

import os.path as osp

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

# Save checkpoints
!cp -r /content/mmdetection/working/tood /content/drive/MyDrive/model

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2022-05-14 16:24:25,254 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-05-14 16:24:25,403 - mmdet - INFO - Start running, host: root@5e72e12bab90, work_dir: /content/mmdetection/working/tood
2022-05-14 16:24:25,404 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) SetEpochInfoHook                   
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook      

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!


2022-05-14 16:25:34,896 - mmdet - INFO - Epoch [1][100/1352]	lr: 2.500e-03, eta: 7:48:29, time: 0.695, data_time: 0.029, memory: 6575, loss_cls: 28.6118, loss_bbox: 1.1586, loss: 29.7704, grad_norm: 474.2566
2022-05-14 16:26:43,122 - mmdet - INFO - Epoch [1][200/1352]	lr: 2.500e-03, eta: 7:43:08, time: 0.682, data_time: 0.007, memory: 6575, loss_cls: 0.4952, loss_bbox: 0.8808, loss: 1.3760, grad_norm: 8.7513
2022-05-14 16:27:49,539 - mmdet - INFO - Epoch [1][300/1352]	lr: 2.500e-03, eta: 7:36:32, time: 0.664, data_time: 0.006, memory: 6575, loss_cls: 0.4704, loss_bbox: 0.4292, loss: 0.8996, grad_norm: 9.8385
2022-05-14 16:28:56,927 - mmdet - INFO - Epoch [1][400/1352]	lr: 2.500e-03, eta: 7:34:19, time: 0.674, data_time: 0.007, memory: 6575, loss_cls: 0.4289, loss_bbox: 0.1748, loss: 0.6037, grad_norm: 5.7854
2022-05-14 16:30:05,857 - mmdet - INFO - Epoch [1][500/1352]	lr: 2.500e-03, eta: 7:34:35, time: 0.689, data_time: 0.007, memory: 6575, loss_cls: 0.4299, loss_bbox: 0.1987, loss: 0.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 298/298, 9.0 task/s, elapsed: 33s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 298/298, 9.0 task/s, elapsed: 33s, ETA:     0s

2022-05-15 00:20:14,228 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.25s).
Accumulating evaluation results...


2022-05-15 00:20:14,610 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.912
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.912
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.912
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.912
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.993
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.993
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.993
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=

DONE (t=0.12s).
